In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    boards = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
                                 board.flatten().astype(float))
        validGuessesSum = sum(validGuesses)
        if validGuessesSum > 0:
            # encode counts as one hot
            countsOneHot = np.zeros((counts.size,10))
            countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
            batch_xs.append(countsOneHot.flatten())
            batch_ys.append(validGuesses / validGuessesSum)
            boards.append(board)
    return (np.asarray(batch_xs), np.asarray(batch_ys), boards)

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
W = tf.Variable(tf.truncated_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.truncated_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
y_image = tf.reshape(y, [-1, rows, cols, 1])

# First convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(y_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer
W_fc1 = weight_variable([(rows // 4) * (cols // 4) * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, (rows // 4) * (cols // 4) * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([1024, size*2])
b_fc2 = bias_variable([size*2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [10]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [11]:
# Loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y_conv))

In [12]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

with tf.name_scope('W_conv1_reshape'):
    image_shaped_W_conv1 = tf.reshape(W_conv1, [-1, 5*5, 32, 1])
    tf.summary.image('W_conv1', image_shaped_W_conv1, 1000)

with tf.name_scope('W_conv2_reshape'):
    image_shaped_W_conv2 = tf.reshape(W_conv2, [-1, 5*32, 5*64, 1])
    tf.summary.image('W_conv2', image_shaped_W_conv2, 1000)

with tf.name_scope('W_fc1_reshape'):
    image_shaped_W_fc1 = tf.reshape(W_fc1, [-1, (rows // 4) * (cols // 4) * 64, 1024, 1])
    tf.summary.image('W_fc1', image_shaped_W_fc1, 1000)

with tf.name_scope('b_fc1_reshape'):
    image_shaped_b_fc1 = tf.reshape(b_fc1, [-1, 32, 32, 1])
    tf.summary.image('b_fc1', image_shaped_b_fc1, 1000)

with tf.name_scope('W_fc2_reshape'):
    image_shaped_W_fc2 = tf.reshape(W_fc2, [-1, 1024, size*2, 1])
    tf.summary.image('W_fc2', image_shaped_W_fc2, 1000)

with tf.name_scope('b_fc2_reshape'):
    image_shaped_b_fc2 = tf.reshape(b_fc2, [-1, rows*2, cols, 1])
    tf.summary.image('b_fc2', image_shaped_b_fc2, 1000)

_ = tf.summary.scalar('loss', cross_entropy)

In [13]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [14]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines7/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [15]:
tf.global_variables_initializer().run()

In [18]:
# Restore model?
saver.restore(sess, savePath + "model-500")

INFO:tensorflow:Restoring parameters from ./saves.tf.Mines7/(8, 8)/model-500


In [17]:
# Train
for iteration in range(500,100001):
    batch_xs, batch_ys, _ = next_training_batch(10000)
    summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                 feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 0.5})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 10 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-07 20:18:02: Loss at step 0: 4.85195
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-0
2017-11-07 20:18:10: Loss at step 1: 4.85106
2017-11-07 20:18:19: Loss at step 2: 4.84994
2017-11-07 20:18:28: Loss at step 3: 4.84887
2017-11-07 20:18:37: Loss at step 4: 4.84681
2017-11-07 20:18:46: Loss at step 5: 4.84455
2017-11-07 20:18:55: Loss at step 6: 4.84051
2017-11-07 20:19:04: Loss at step 7: 4.83536
2017-11-07 20:19:12: Loss at step 8: 4.83179
2017-11-07 20:19:21: Loss at step 9: 4.83434
2017-11-07 20:19:30: Loss at step 10: 4.83467
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-10
2017-11-07 20:19:39: Loss at step 11: 4.8316
2017-11-07 20:19:47: Loss at step 12: 4.82863
2017-11-07 20:19:56: Loss at step 13: 4.82964
2017-11-07 20:20:05: Loss at step 14: 4.82917
2017-11-07 20:20:14: Loss at step 15: 4.82945
2017-11-07 20:20:22: Loss at step 16: 4.83035
2017-11-07 20:20:31: Loss at step 17: 4.82776
2017-11-07 20:20:40: Loss at step 18: 4.82784
2017-11-07 20:20:49: Loss at

2017-11-07 20:41:16: Loss at step 159: 4.63168
2017-11-07 20:41:25: Loss at step 160: 4.63067
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-160
2017-11-07 20:41:34: Loss at step 161: 4.6262
2017-11-07 20:41:43: Loss at step 162: 4.62182
2017-11-07 20:41:51: Loss at step 163: 4.62726
2017-11-07 20:42:00: Loss at step 164: 4.62038
2017-11-07 20:42:09: Loss at step 165: 4.62339
2017-11-07 20:42:18: Loss at step 166: 4.62122
2017-11-07 20:42:27: Loss at step 167: 4.62079
2017-11-07 20:42:36: Loss at step 168: 4.61773
2017-11-07 20:42:45: Loss at step 169: 4.61943
2017-11-07 20:42:54: Loss at step 170: 4.61721
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-170
2017-11-07 20:43:02: Loss at step 171: 4.62093
2017-11-07 20:43:11: Loss at step 172: 4.61743
2017-11-07 20:43:20: Loss at step 173: 4.61716
2017-11-07 20:43:29: Loss at step 174: 4.61392
2017-11-07 20:43:38: Loss at step 175: 4.6135
2017-11-07 20:43:46: Loss at step 176: 4.6151
2017-11-07 20:43:55: Loss at step 177: 4.6133

2017-11-07 21:04:02: Loss at step 315: 4.41465
2017-11-07 21:04:11: Loss at step 316: 4.41351
2017-11-07 21:04:20: Loss at step 317: 4.41793
2017-11-07 21:04:28: Loss at step 318: 4.41027
2017-11-07 21:04:37: Loss at step 319: 4.41155
2017-11-07 21:04:46: Loss at step 320: 4.41138
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-320
2017-11-07 21:04:55: Loss at step 321: 4.40737
2017-11-07 21:05:03: Loss at step 322: 4.4032
2017-11-07 21:05:12: Loss at step 323: 4.40479
2017-11-07 21:05:21: Loss at step 324: 4.39985
2017-11-07 21:05:29: Loss at step 325: 4.40272
2017-11-07 21:05:38: Loss at step 326: 4.39545
2017-11-07 21:05:47: Loss at step 327: 4.39984
2017-11-07 21:05:56: Loss at step 328: 4.39423
2017-11-07 21:06:04: Loss at step 329: 4.39469
2017-11-07 21:06:13: Loss at step 330: 4.39125
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-330
2017-11-07 21:06:22: Loss at step 331: 4.39337
2017-11-07 21:06:30: Loss at step 332: 4.3865
2017-11-07 21:06:39: Loss at step 333: 4.393

2017-11-07 21:26:46: Loss at step 471: 4.1804
2017-11-07 21:26:54: Loss at step 472: 4.17923
2017-11-07 21:27:03: Loss at step 473: 4.18266
2017-11-07 21:27:12: Loss at step 474: 4.17464
2017-11-07 21:27:21: Loss at step 475: 4.1871
2017-11-07 21:27:29: Loss at step 476: 4.18846
2017-11-07 21:27:38: Loss at step 477: 4.17504
2017-11-07 21:27:47: Loss at step 478: 4.17595
2017-11-07 21:27:56: Loss at step 479: 4.18348
2017-11-07 21:28:05: Loss at step 480: 4.17848
Model saved in file: ./saves.tf.Mines7/(8, 8)/model-480
2017-11-07 21:28:14: Loss at step 481: 4.16906
2017-11-07 21:28:23: Loss at step 482: 4.17389
2017-11-07 21:28:31: Loss at step 483: 4.17399
2017-11-07 21:28:40: Loss at step 484: 4.17678
2017-11-07 21:28:49: Loss at step 485: 4.16819
2017-11-07 21:28:58: Loss at step 486: 4.17196
2017-11-07 21:29:06: Loss at step 487: 4.17428
2017-11-07 21:29:15: Loss at step 488: 4.16619
2017-11-07 21:29:24: Loss at step 489: 4.17922
2017-11-07 21:29:34: Loss at step 490: 4.16729
Model 

KeyboardInterrupt: 

In [ ]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(10000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys}))

In [19]:
# Run a test
batchSize = 10000
batch_xs, batch_ys,_ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  10000
6615


In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [ ]:
np.save("./W", sess.run(W))

In [ ]:
np.save("./b", sess.run(b))

In [ ]:
np.savez("./model", sess.run([W,b]))